# Análisis de regresión
"*Todos los modelos están equivocados, pero algunos son útiles" G.E.P. Box (1919-2013)*

## ¿Qué es el análisis de regresión lineal?

El análisis de regresión lineal es una técnica utilizada para modelar la relación entre una variable dependiente y una o más variables independientes. Su objetivo principal es explicar o predecir el comportamiento de la variable dependiente (por ejemplo, las ventas) en función de las variables independientes (por ejemplo, el gasto en publicidad).

### Interpretación de la regresión lineal múltiple
Se asume que esta relación sigue una ecuación lineal conocida como ecuación de regresión. En su forma básica la ecuación o modelo de regresión múltiple se expresa como:
$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 +...+ \beta_k x_k + \varepsilon
$$
donde:
- $y$ es la variable dependiente, aquella que se desea explicar o predecir
- $x_1, x_2,...,x_n$ son las variable independientes, que sirven para explicar $y$. Si solo hay una variable independiente se conoce como *regresión lineal simple*, si hay dos o más, se trata de una *regresión lineal múltiple*.
- $\beta_0$ es el término de intersección (también llamado constante o intercepto), que representa el valor de $y$ si todas las variables independientes fueran igual a cero. Este valor solo tiene interpretación válida en algunos contextos.
- $\beta_1, \beta_2,..., \beta_k$ son los coeficientes de regresión, que indican el cambio estimado en la variable dependiente por unidad de cambio en la respectiva variable independiente, manteniendo todo lo demás constante.
- $\varepsilon$ es el término de error, que captura la variabilidad en $y$ que no es explicada por las variables independientes. Cuando se analiza un modelo ajustado a datos específicos, es más preciso referirse a este error como *residual*.

La interpretación de los resultados permite:
- Evaluar si la variable independiente tiene un efecto significativo sobre la variable dependiente.
- Medir la magnitud y dirección del efecto de la variable independiente sobre la dependiente.
- Realizar predicciones basadas en las variables independientes.

El método más común para estimar los coeficientes de la ecuación de regresión es el de mínimos cuadrados ordinarios (MCO), el cual minimiza la suma de los errores al cuadrado para encontrar la mejor línea de ajuste.

Ejemplo: https://phet.colorado.edu/sims/html/least-squares-regression/latest/least-squares-regression_en.html


### Supuestos básicos del modelo
Las estimaciones de los coeficientes de regresión serán óptimas según el teorema de Gauss-Markov, siempre que el término de error cumpla con los siguientes supuestos:
- El error tiene media igual a cero. Esto implica que, en promedio, los errores no sesgan las predicciones del modelo.
- Los errores son independientes. No existe correlación entre los errores de diferentes observaciones.
- Homocedasticidad: La varianza del error es constante para todos los valores de las variables independientes.
- Distribución normal del error (en caso de inferencia estadística). Este supuesto es requerido para realizar pruebas de hipótesis y construir intervalos de confianza.

Estas condiciones se pueden resumir en la notación $\varepsilon$~N<sub>iid</sub> (0,σ<sup>2</sup>). Si se cumplen estas condiciones, los estimadores de los coeficientes de regresión serán los mejores estimadores insesgados de mínima varianza (*MELI* por las siglas en español; *BLUE* por las siglas en inglés de *Best Linear Unbiased Estimators*). 

## Análisis de regresión
Utilizaremos un modelo de regresión para estimar el precio de una casa en una zona de la ciudad. Como primer paso importamos las bibliotecas a utilizar.

In [ ]:
# Importar bibliotecas
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Características de la muestra
Existen diversas recomendaciones para determinar el tamaño de muestra adecuado en un análisis de regresión. Para generalizar los resultados, se sugiere contar con al menos 5 observaciones por cada variable independiente, aunque lo ideal es tener entre 15 y 20 observaciones por variable independiente (Hair, Black, Babin, & Anderson, 2009).

Por otro lado, Green (1991) argumenta que no es adecuado establecer un tamaño de muestra fijo (por ejemplo, 100 observaciones). En su lugar, propone las siguientes reglas:
- Para evaluar un modelo de regresión, se recomienda un mínimo de $50 + 8k$ observaciones, donde k es el número de variables independientes.
- Para evaluar los coeficientes individuales de cada variable independiente, se sugiere un mínimo de $104 + k$ observaciones.

Más allá del tamaño de la muestra, es fundamental asegurarse de que esta sea *representativa* de la población de interés para garantizar la validez de los resultados.

In [ ]:
# Importar el archivo "data/casas.xlsx"
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/data/casas.xlsx')
# Para este ejemplo seleccionamos casas (tipo = 0 )
df = df[df['tipo']==0] 

In [ ]:
# Revisar las variables y el número de observaciones
df.info()

In [ ]:
# Revisar las primeras filas del DataFrame
df.head(3)

### Características de las variables
En un modelo de regresión lineal múltiple, la variable dependiente debe ser cuantitativa, es decir, debe tener una escala de medición de intervalo o razón. Además, al menos una de las variables independientes también debe ser cuantitativa.

In [ ]:
# Identifica las variables cuantitativasy obtener la estadística descriptiva
var_cont = df[['preciomillones', 'recamaras', 'baños', 'construccion']]
var_cont.describe().T

Un aspecto clave en la selección de las variables independientes es evitar la multicolinealidad, que ocurre cuando una combinación lineal de dos o más variables contiene la misma información que otra variable independiente. Aunque existen técnicas formales para detectar la multicolinealidad (como el factor de inflación de la varianza, VIF), un primer criterio práctico es evitar correlaciones altas entre variables independientes, por ejemplo, valores de $r>0.90$

In [ ]:
# Obtenemos la matriz de correlaciones
var_cont.corr()

La función `sns.pairplot` de *Seaborn* es útil para visualizar las relaciones entre pares de variables en un DataFrame. Sus principales parámetros son:
- `data`: Especifica el DataFrame con las variables a graficar.
- `corner`: Si se establece en True, se muestra solo la matriz triangular inferior, lo que reduce la redundancia en la visualización.
- `kind`: Define el tipo de gráfico. Por defecto, usa "scatter", pero también puede configurarse como "reg" para incluir una recta de regresión en cada gráfico de dispersión.
- `markers`: Permite definir el tipo de marcador en el gráfico. Por ejemplo, usar '+' en lugar de '.' cambia el estilo de los puntos.
- `height`: Controla la altura (en pulgadas) de cada gráfico dentro de la matriz.

In [ ]:
# Graficamos las relaciones entre variables
sns.pairplot(var_cont, corner=True, kind='reg', markers='+', height=1.5);

### Estimación del modelo (con StatsModels)
Utilizaremos primero la biblioteca *StatsModels* para realizar el análisis de regresión

In [ ]:
# Importar la librería
import statsmodels.api as sm

Es recomendable crear un DataFrame `X` que contenga las variables independientes y otro `y` con la variable dependiente.

Para seleccionar las variables independientes, primero es importante evaluar cuáles pueden ser relevantes para el modelo. Si hay muchas variables independientes, se puede aplicar un *Análisis Factorial* u otras técnicas de reducción de dimensionalidad, como *Análisis de Componentes Principales (PCA)*.

En este ejemplo, comenzaremos con la variable "construcción" como predictor. Para utilizar la función de StatsModels, es necesario agregar manualmente una columna de unos (`1`) al DataFrame X, lo que representa la constante en el modelo de regresión. Esto se hace con `sm.add_constant()`.

In [ ]:
# Creamos un DataFrame con las v. indep. y la v. dependiente
X = df[['construccion', 'baños']]
X = sm.add_constant(X)

y = df['preciomillones']

Corremos el modelo de regresión mediante mínimos cuadrados ordinarios:  
`regresion = sm.OLS(y, X).fit()`

In [ ]:
regresion = sm.OLS(y, X).fit()

El resultado se muestra con la instrucción
`summary()`.

In [ ]:
regresion.summary()

In [ ]:
# Mostrar coeficientes de regresión
regresion.params

In [ ]:
# Guardar residuales
residuales = regresion.resid

In [ ]:
# Obtener valores estimados
y_hat = regresion.predict(X)

Ejemplo de pronóstico

In [ ]:
# Pronóstico
nuevos_valores = [1,500,2]
regresion.predict(nuevos_valores)

## Interpretación de los resultados
Al ajustar un modelo de regresión con StatsModels, se generan diversas métricas que permiten evaluar su calidad y ajuste. A continuación, se explican algunas de las más relevantes:
- *R-squared (R²)*: Indica el porcentaje de la variabilidad de la variable dependiente que es explicada por el modelo. Valores más cercanos a 1 son mejores.
- *Adj. R-squared (R² ajustado)*: Penaliza la inclusión de variables no significativas, por lo que se usa para comparar modelos con diferente número de variables independientes.
- *AIC (Criterio de Información de Akaike) / BIC (Criterio de Información Bayesiano)*: Son medidas relativas para comparar modelos de regresión múltiple. Valores menores indican modelos más eficientes en términos de ajuste y simplicidad.
- *F-statistic*: Evalúa la significancia global del modelo. Su hipótesis nula es que todos los coeficientes de regresión son cero (es decir, que el modelo no explica la variable dependiente). Un p-valor bajo sugiere que al menos una variable independiente es significativa.
- *Skew (Sesgo)*: Mide la asimetría de la distribución de los residuales. Valores cercanos a 0 indican una distribución simétrica.
- *Kurtosis*: Indica qué tan puntiaguda o achatada es la distribución de los residuales en comparación con una normal.
- *Prob(Omnibus) y Jarque-Bera (JB)*: Son pruebas para evaluar la normalidad de los residuales. Un p-valor alto sugiere que los residuales siguen una distribución normal, lo que es deseable.
- *Durbin-Watson*: Detecta autocorrelación en los errores. Valores cercanos a 2 son deseables. Valores entre 0 y 2 indican autocorrelación positiva. Valores entre 2 y 4 indican autocorrelación negativa.
- *Cond. No. (Número de condición)*: Evalúa posibles problemas de multicolinealidad.

### Ajuste del modelo
La métrica más utilizada para medir el ajuste del modelo es el *coeficiente de determinación* (R<sup>2</sup>) que mide la proporción de la variación de la variable dependiente que es explicada por el modelo. 

En un modelo de regresión lineal, la variación total de los datos se mide con sumas de cuadrados:
$$
SS_{total} = SS_{modelo} + SS_{residual}
$$
Donde:

$SS_{total} = \sum_{i = 1}^{n} (y-\bar{y})^2$ es la suma de cuadrados total  
$SS_{modelo} = \sum_{i = 1}^{n} (\hat{y}-\bar{y})^2$ es la suma de cuadrados del modelo también conocida como suma de cuadrados explicada  
$SS_{residual} = \sum_{i = 1}^{n} (y-\hat{y})^2$ es la suma de cuadrados del residual o suma de cuadrados del error

El coeficiente de determinación es equivalente a

$$
R^2 = SS_{modelo} / SS_{total}
$$

La R cuadrada toma valores entre 0 y 1, donde valores más grandes indican un mejor ajuste. Sin embargo, no existen criterios únicos de qué valor de R cuadrada es aceptable, sino que ello depende de las características del estudio.

## Evaluación de supuestos
### Homocedasticidad
Uno de los supuestos del modelo de regresión es que la varianza de los residuales se mantiene constante (homocedasticidad). El cumplimiento de este supuesto se puede explorar en la gráfica del residual estandarizado versus los valores pronosticados. 
- Un patrón aleatorio en la dispersión de los residuales sugiere homocedasticidad.
- Un patrón de dispersión en forma de abanico (creciente o decreciente) indicaría una posible violación a este supuesto, lo que sugiere heterocedasticidad.

Existen varias pruebas para evaluar la presencia de heterocedasticidad:
- *Breusch-Pagan*: Evalúa la relación entre los residuales al cuadrado y las variables independientes. Un resultado significativo indica heterocedasticidad..
- *Prueba de White*: Similar a la Breush-Pagan, pero más general, ya que considera términos no lineales y la correlación serial.
- *Prueba de Goldfeld-Quandt*: Divide los datos en dos grupos según los valores de una variable independiente y compara las varianzas de los residuales en ambos grupos. Una diferencia significativa sugiere heterocedasticidad.

In [ ]:
# Gráfico de residuales
plt.figure(figsize=(6, 3))
plt.scatter(y_hat, residuales, marker='.', color='b')
plt.xlabel('y hat')
plt.ylabel('residuales');

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Asumiendo que ya se tiene el modelo y los residuales
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(residuales, X)

print("Estadístico LM:", lm)
print("Valor p del estadístico LM:", lm_p_value)
print("Estadístico F:", fvalue)
print("Valor p del estadístico F:", f_p_value)

Si el p-valor es menor que 0.05 es evidencia de la presencia de heteroscedasticidad

*¿Qué hacer si se detecta heteroscedasticidad?*
- Explorar transformaciones de variables (p. ej. logaritmos)
- Utilizar otros modelos de regresión, como Mínimos Cuadrados Ponderados (WLS) que asigna pesos inversamente proporcionales a la varianza de los errores
- Crear modelos separados.

### Normalidad
La normalidad en la distribución de los errores y su media se puede verificar en el histograma de los residuales y una prueba de normalidad (como Shapiro-Wilks o  Kolmogorov-Smirnov). En la prueba de normalidad lo deseable es obtener un p-valor mayor a 0.05 (no se rechaza la hipótesis nula de normalidad en los datos). 

El reporte de salida de StatsModelo muestra el p-valor de las pruebas omnibus y de Jarque-Bera, ambas basadas en el sesgo y la curtosis.

In [ ]:
plt.figure(figsize=(4, 3)) 
sns.histplot(residuales, color='b');

In [ ]:
from scipy.stats import shapiro
stat, p_value = shapiro(residuales)
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

*¿Qué hacer si se detecta no normalidad en los residuales?*

- Revisar outliers o valores influyentes.
- Explorar transformaciones de variables (p. ej. logaritmos)
- Utilizar otros modelos de regresión.
- Explorar variables no incluidas en el modelo de regresión.
- Utilizar bootstrapping para las estimaciones de intervalos.

### Independencia en los errores
Los errores deben ser independientes en una regresión, esto es, no relacionados entre sí.  Generalmente esto implica evaluar la autocorrelación (correlación entre valores adyacentes) y se lleva a cabo mediante la prueba de Durbin-Watson (solamente en series de tiempo). En esta prueba, el estadístico de prueba puede tomar valores entre 0 y 4. Si el estadístico es cercano a 2 entonces los residuales no están correlacionados. Valores menores a 1 o mayores a 3 son señal de un problema serio de autocorrelación. 

*¿Qué hacer si se detecta autocorrelación en los residuales?*
- Explorar modelos específicos para series temporales (ARIMA, SARIMA)

### Multicolinealidad
La multicolinealidad se presenta cuando existe una fuerte correlación entre dos o más variables independientes o una combinación lineal de ellas. La multicolinealidad puede llevar a tener un modelo estadísticamente significativo, pero coeficientes de regresión no significativos.

Para evaluar la multicolinealidad:
- Revisar la matriz de correlaciones de las variables independientes para identificar correlaciones arriba de 0.90
- Revisar el número de condición, valores mayores a 30 indicarían un problema grave
- Calcular el Factor de Inflación de Varianza de cada variable, valores mayores a 10 indicarían un problema grave

*¿Qué hacer si se detecta multicolinealidad?*
- Regularizar (regresión Lasso o Ridge).
- Combinar variables.
- Eliminar variables redundantes.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[['construccion', 'baños']]
X = sm.add_constant(X)

# Calcular valores VIF para cada variable
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data

## Extra: uso de scikit-learn

In [ ]:
# Importar la biblioteca
from sklearn.linear_model import LinearRegression

Con *SciKit-Learn' no se agrega la columna de constantes como en StatsModel

In [ ]:
X = df[['construccion']]
y = df['preciomillones']
model = LinearRegression().fit(X, y)

In [ ]:
# Obtener los coeficientes de regresión
model.intercept_, model.coef_

In [ ]:
# Obtener y estimada (y hat)
y_hat = model.predict(X)

In [ ]:
# Obtener residuales
residuales = (y - y_hat).values

In [ ]:
# Calcular R cuadrada
from sklearn.metrics import r2_score
r2_score(y, y_hat)